In [1]:
import Utils as u

In [2]:
query_texto = "leche de vaca"
query_embedding = u.get_embedding(query_texto)
print(query_embedding)

[-0.012594055384397507, -0.008182577788829803, -0.005999479442834854, -0.0380861833691597, -0.008641837164759636, -0.012587586417794228, -0.03459322452545166, -0.033092547208070755, -0.012354723177850246, -0.043545547872781754, 0.00999374222010374, 0.030945023521780968, -0.009916120208799839, -0.0002933033974841237, -0.02346750535070896, 0.017037874087691307, 0.019198333844542503, -0.023842673748731613, 0.03855191171169281, -0.008176109753549099, -0.00835722591727972, 0.011132187210023403, 0.021824520081281662, -0.016352219507098198, -0.006597809959203005, -0.007147627416998148, 0.01179196871817112, -0.011539699509739876, 0.005608138628304005, -0.006449035834521055, 0.016274597495794296, -0.002171779051423073, -0.011436204425990582, 0.011041629128158092, -0.025175172835588455, -0.03063453733921051, -0.002470944542437792, -0.01720605418086052, 0.0037161195650696754, -0.003300522221252322, -0.004964528605341911, 0.010278353467583656, -0.0050259786657989025, 0.0023480441886931658, -0.0186

In [3]:
## Ver un registro en ChromaDB
# u.collection.peek(limit=1)

In [3]:
productos_relacionados = u.query_products_chromadb(query_embedding=query_embedding, k=3)
print(productos_relacionados)

model = "gp-3.5-turbo"

formatted_response = u.format_response_with_openai(productos_relacionados, u.gpt_client, model)
# Imprime el resultado formateado
print("\nRespuestas formateadas:")
for idx, response in enumerate(formatted_response, 1):
    print(f"\nProducto {idx}:")
    print(response)
    print("-" * 50)

{'productos': [{'producto': 'Leche sin Lactosa LAIVE Light Botella 390g', 'descripcion': 'plazaVea', 'precio': 4.7, 'market': 'plazaVea', 'url': 'https://www.plazavea.com.pe/leche-sin-lactosa-laive-light-botella-390g/p', 'categorias': 'Lácteos y Huevos, Leche, Leche Evaporada y Mezclas Lácteas'}, {'producto': 'Leche sin Lactosa LAIVE Botella 390g', 'descripcion': 'plazaVea', 'precio': 4.4, 'market': 'plazaVea', 'url': 'https://www.plazavea.com.pe/leche-sin-lactosa-laive-botella-390g/p', 'categorias': 'Lácteos y Huevos, Leche, Leche Evaporada y Mezclas Lácteas'}, {'producto': 'Leche Fermentada GLORIA Yofresh Sabor a Vainilla Botella 320g', 'descripcion': 'plazaVea', 'precio': 3.2, 'market': 'plazaVea', 'url': 'https://www.plazavea.com.pe/leche-fermentada-gloria-yofresh-sabor-a-vainilla-botella-320g/p', 'categorias': 'Lácteos y Huevos, Yogurt, Yogurt Bebible'}], 'distances': [0.2807267904281616, 0.28403016924858093, 0.28506144881248474]}

Respuestas formateadas:

Producto 1:
Error proces